In [1]:
import cv2
import numpy as np
import os

dataset_path = "dataset"
haar_file = "haarcascade_frontalface_default.xml"
recognizer = cv2.face.LBPHFaceRecognizer_create()

faces = []
labels = []

for folder_name in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder_name)
    if not os.path.isdir(folder_path):
        continue

    roll_no = folder_name.split('_')[0]  # Use only roll number
    try:
        label = int(roll_no)
    except:
        continue

    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            faces.append(img)
            labels.append(label)

print(f"✅ Collected {len(faces)} images from {len(set(labels))} students.")
recognizer.train(faces, np.array(labels))
recognizer.save("trainer.yml")
print("🎉 Model saved to trainer.yml")


✅ Collected 60 images from 2 students.
🎉 Model saved to trainer.yml
